![alt text](../img/header.jpg)

# Groundwater Transport

In [ ]:
import os
import sys
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy

import config

print(sys.version)
print('python executable: {}'.format(sys.executable))
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

In [ ]:
os.path.isfile(config.mf6betaexe)
errmsg = ('could not find mf6beta. '
          'download from here: '
          'https://github.com/langevin-usgs/mf6flopy2019_classrepo/releases')
assert os.path.isfile(config.mf6betaexe), errmsg

In [ ]:
# Be careful here!!
#flopy.mf6.utils.generate_classes(branch='mf6beta')

In [ ]:
def get_model(workspace, model_name='mymodel'):
    nlay, nrow, ncol = 40, 12, 30
    nper = 1
    perlen = [400]
    nstp = [400]
    tsmult = [1.]
    steady = [True]
    delr = 3.0
    delc = 3.0
    top = 0.
    delz = 0.05
    botm =  np.arange(-delz, -nlay*delz - delz, -delz)
    strt = 0.
    hnoflo = 1e30
    hdry = -1e30
    hk = 0.0125 / delz
    laytyp = 0
    diffc = 0.
    alphal = 0.6
    alphath = 0.03
    alphatv = 0.006
    porosity = 0.25
    #ss = 0.
    #sy = 0.1

    nouter, ninner = 100, 300
    hclose, rclose, relax = 1e-4, 1e-3, 0.97

    tdis_rc = []
    for i in range(nper):
        tdis_rc.append((perlen[i], nstp[i], tsmult[i]))

    name = model_name

    # build MODFLOW 6 files
    ws = workspace
    sim = flopy.mf6.MFSimulation(sim_name=name, version='mf6',
                                 exe_name=config.mf6betaexe,
                                 sim_ws=ws)
    # create tdis package
    tdis = flopy.mf6.ModflowTdis(sim, time_units='DAYS',
                                 nper=nper, perioddata=tdis_rc)

    # create gwf model
    gwfname = 'gwf_' + name
    gwf = flopy.mf6.MFModel(sim, model_type='gwf6', modelname=gwfname,
                            model_nam_file='{}.nam'.format(gwfname))

    # create iterative model solution and register the gwf model with it
    imsgwf = flopy.mf6.ModflowIms(sim, print_option='SUMMARY',
                                  outer_hclose=hclose,
                                  outer_maximum=nouter,
                                  under_relaxation='NONE',
                                  inner_maximum=ninner,
                                  inner_hclose=hclose, rcloserecord=rclose,
                                  linear_acceleration='CG',
                                  scaling_method='NONE',
                                  reordering_method='NONE',
                                  relaxation_factor=relax,
                                  filename='{}.ims'.format(gwfname))
    sim.register_ims_package(imsgwf, [gwf.name])

    dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                                  delr=delr, delc=delc,
                                  top=top, botm=botm,
                                  idomain=np.ones((nlay, nrow, ncol), dtype=np.int),
                                  filename='{}.dis'.format(gwfname))

    # initial conditions
    ic = flopy.mf6.ModflowGwfic(gwf, strt=strt,
                                filename='{}.ic'.format(gwfname))

    # node property flow
    npf = flopy.mf6.ModflowGwfnpf(gwf, save_flows=False,
                                  save_specific_discharge=True,
                                  icelltype=laytyp,
                                  k=hk,
                                  k33=hk)

    # chd files
    chdlist = []
    j = ncol - 1
    for k in range(nlay):
        for i in range(nrow):
            chdlist.append([(k, i, j), 0.])
    chd = flopy.mf6.ModflowGwfchd(gwf,
                                  stress_period_data=chdlist,
                                  save_flows=False,
                                  pname='CHD-1')

    # wel files
    wellist = []
    j = 0
    qwell = 0.1 * delz * delc * porosity
    for k in range(nlay):
        for i in range(nrow):
            wellist.append([(k, i, j), qwell, 0.])
    wellist.append([(0, 0, 7), 1.e-6, 2.5e6])  # source well

    wel = flopy.mf6.ModflowGwfwel(gwf,
                                  print_input=True,
                                  print_flows=True,
                                  stress_period_data=wellist,
                                  save_flows=False,
                                  auxiliary='CONCENTRATION',
                                  pname='WEL-1')

    # output control
    oc = flopy.mf6.ModflowGwfoc(gwf,
                                budget_filerecord='{}.cbc'.format(gwfname),
                                head_filerecord='{}.hds'.format(gwfname),
                                headprintrecord=[
                                    ('COLUMNS', 10, 'WIDTH', 15,
                                     'DIGITS', 6, 'GENERAL')],
                                saverecord=[('HEAD', 'LAST')],
                                printrecord=[('HEAD', 'LAST'),
                                             ('BUDGET', 'LAST')])

    # create gwt model
    gwtname = 'gwt_' + name
    gwt = flopy.mf6.MFModel(sim, model_type='gwt6', modelname=gwtname,
                            model_nam_file='{}.nam'.format(gwtname))

    # create iterative model solution and register the gwt model with it
    imsgwt = flopy.mf6.ModflowIms(sim, print_option='SUMMARY',
                                  outer_hclose=hclose,
                                  outer_maximum=nouter,
                                  under_relaxation='NONE',
                                  inner_maximum=ninner,
                                  inner_hclose=hclose, rcloserecord=rclose,
                                  linear_acceleration='BICGSTAB',
                                  scaling_method='NONE',
                                  reordering_method='NONE',
                                  relaxation_factor=relax,
                                  filename='{}.ims'.format(gwtname))
    sim.register_ims_package(imsgwt, [gwt.name])

    dis = flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
                                  delr=delr, delc=delc,
                                  top=top, botm=botm,
                                  idomain=1,
                                  filename='{}.dis'.format(gwtname))

    # initial conditions
    strt = np.zeros((nlay, nrow, ncol))
    strt[0, 0, 0] = 0.
    ic = flopy.mf6.ModflowGwtic(gwt, strt=strt,
                                filename='{}.ic'.format(gwtname))

    # advection
    adv = flopy.mf6.ModflowGwtadv(gwt, scheme='TVD',
                                  filename='{}.adv'.format(gwtname))

    # dispersion
    dsp = flopy.mf6.ModflowGwtdsp(gwt, xt3d=None, diffc=diffc,
                                  alh=alphal, alv=alphal,
                                  ath1=alphath, ath2=alphatv,
                                  filename='{}.dsp'.format(gwtname))

    # storage
    sto = flopy.mf6.ModflowGwtsto(gwt, porosity=porosity,
                                filename='{}.sto'.format(gwtname))

    # sources
    sourcerecarray = [('WEL-1', 1, 'CONCENTRATION')]
    ssm = flopy.mf6.ModflowGwtssm(gwt, sources=sourcerecarray,
                                filename='{}.ssm'.format(gwtname))

    # output control
    oc = flopy.mf6.ModflowGwtoc(gwt,
                                budget_filerecord='{}.cbc'.format(gwtname),
                                concentration_filerecord='{}.ucn'.format(gwtname),
                                concentrationprintrecord=[
                                    ('COLUMNS', 10, 'WIDTH', 15,
                                     'DIGITS', 6, 'GENERAL')],
                                saverecord=[('CONCENTRATION', 'ALL')],
                                printrecord=[('CONCENTRATION', 'LAST'),
                                             ('BUDGET', 'LAST')])

    # GWF GWT exchange
    gwfgwt = flopy.mf6.ModflowGwfgwt(sim, exgtype='GWF6-GWT6',
                                     exgmnamea=gwfname, exgmnameb=gwtname,
                                     filename='{}.gwfgwt'.format(name))

    return sim

In [ ]:
workspace = 'ex0x-completed-transport'
sim = get_model(workspace)

In [ ]:
# write the input files
sim.write_simulation()

In [ ]:
# run the simulation
sim.run_simulation()

In [ ]:
print(sim.model_names)
gwfname = sim.model_names[0]
gwtname = sim.model_names[1]
gwf = sim.get_model(gwfname)
gwt = sim.get_model(gwtname)

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = plt.subplot(1, 1, 1, aspect='equal')
pmv = flopy.plot.PlotMapView(model=gwf)
pmv.plot_grid()

In [ ]:
fname = os.path.join(workspace, gwtname + '.ucn')
concobj = flopy.utils.HeadFile(fname, text='CONCENTRATION')
conc = concobj.get_data()
time_series = concobj.get_ts([(0, 0, 10), (20, 0, 10)])

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = plt.subplot(1, 1, 1, aspect='equal')
pmv = flopy.plot.PlotMapView(model=gwf, layer=3)
pmv.plot_grid(color='.5')
conc2plot = np.ma.masked_less_equal(conc, 1)
pmv.plot_array(conc2plot, alpha=0.5, cmap='jet')
pmv.contour_array(conc, levels=[.1, 1, 10, 100, 1000])

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = plt.subplot(1, 1, 1)
ax.plot(time_series[:, 0], time_series[:, 1:])
ax.set_xlabel('TIME')
ax.set_ylabel('CONCENTRATION')